In [ ]:
# .env 파일을 읽어서 환경변수로 설정
from dotenv import load_dotenv

# 토큰 정보로드
load_dotenv()

In [ ]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("Runnable")

## Langchain에서 데이터를 효과적으로 전달하는 방법
1. RunnablePassthrough
2. RunnableParallel
3. RunnableLambda


### 1. `RunnablePassthrough`: 데이터를 그대로 넘겨(통과시켜)주는 역할

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI


# prompt 와 llm 을 생성합니다.
prompt = PromptTemplate.from_template("{word}를 영어로?")
llm = ChatOpenAI(temperature=0)

# chain 을 생성합니다.
chain = prompt | llm
chain

chain 을 `invoke()` 하여 실행할 때는 입력 데이터의 타입은 ***딕셔너리***

In [ ]:
# chain 을 실행합니다.
chain.invoke({"word": '사과'})

But, 1개의 변수만 템플릿에 포함하고 있다면, 값만 전달하는 것도 가능

In [ ]:
# chain 을 실행합니다.
chain.invoke('사과')

In [ ]:
from langchain_core.runnables import RunnablePassthrough

# RunnablePassthrough 는 runnable 객체이며, runnable 객체는 invoke() 메소드를 사용하여 별도 실행이 가능
# RunnablePassthrough()를 사용하여 체인 구성
runnable_chain = {"word": RunnablePassthrough()} | prompt | ChatOpenAI()

# dict 값이 RunnablePassthrough() 로 변경됨
runnable_chain.invoke('인공지능')

### 2. `RunnableParallel`: 여러 작업을 동시에(병렬)로 처리하도록 도와주는 도구

In [11]:
from langchain_core.runnables import RunnableParallel

chain1 = (
    {"country": RunnablePassthrough()}
    | PromptTemplate.from_template("{country} 의 수도는?")
    | ChatOpenAI()
)
chain2 = (
    {"country": RunnablePassthrough()}
    | PromptTemplate.from_template("{country} 의 면적은?")
    | ChatOpenAI()
)

In [ ]:
combined_chain = RunnableParallel(capital=chain1, area=chain2)
combined_chain.invoke("대한민국")

### 3. `RunnableLambda`: 사용자 정의 함수 매핑하도록 도와주는 도구


In [13]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from datetime import datetime
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

def concat_output(text):
    return text['capital'].content + ' ' + text['area'].content

In [ ]:
final_chain = (combined_chain
            | {'info': RunnableLambda(concat_output)} 
            | PromptTemplate.from_template("{info}의 내용을 자연스럽게 다듬고, 이모지를 넣어줘.")
            | ChatOpenAI())

final_chain.invoke("대한민국")

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from datetime import datetime


def get_today(a):
    # 오늘 날짜를 가져오기
    return datetime.today().strftime("%b-%d")


# 오늘 날짜를 출력
get_today(None)

In [16]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

# prompt 와 llm 을 생성합니다.
prompt = PromptTemplate.from_template(
    "{today} 가 생일인 대한민국 유명인 {n} 명을 나열하세요. 생년월일을 표기해 주세요."
)
llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini")

# chain 을 생성합니다.
chain = (
    {"today": RunnableLambda(get_today), "n": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
# 출력
print(chain.invoke(3))

### [실습] Runnnable 세가지를 적절히 사용하여 챗봇 생성

- 위 코드를 활용하여 아래 내용에 맞게 작성하시요.
- RunnableParallel 사용
    - chain1 : `{food}` 재료로 만들 수 있는 메인 디시를 묻는 체인
    - chain2 : `{food}` 재료로 만들 수 있는 후식 디저트를 묻는 체인
- RunnableLambda 사용
    - 두 체인의 결과 값 이어 붙이기 -> `{info}`
    - `{info}`를 바탕으로 이모지를 사용하여 우리 음식점 메뉴 추천 홍보문구를 작성해주는 템플릿


In [ ]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 첫 번째 체인: 재료로 만들 수 있는 메인 디시를 묻는 체인
chain1 = (
)

# 두 번째 체인: 재료로 만들 수 있는 후식 디저트를 묻는 체인
chain2 = (
)

# 두 체인을 병렬로 실행
combined_chain = 

# 두 체인의 결과를 이어 붙이는 함수
def concat_food_info(text):
    return 

# {info}를 바탕으로 이모지를 사용하여 우리 음식점 메뉴 추천 홍보문구를 작성해주는 템플릿
prompt = 

# 최종 체인은 두 체인의 결과를 이어 붙이고 우리 가게 홍보 문구를 생성하는 작업
final_chain = (

)

# 사용자가 생성할 메뉴의 개수를 입력받아 처리
final_output = 

# 결과 출력
print(final_output)


In [ ]:
final_output